# 1. Exploracion inicial 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
data = pd.read_csv('../input/yelow-taxi-2019-01/yellow_tripdata_2019-01.csv')

In [ ]:
data.head()

In [ ]:
data.shape

son 7,6 millones de registros en el periodo de enero del 2019

# Tipo de datos

In [ ]:
data.dtypes


In [ ]:
#2019-01-14 17:24:55
data['tpep_pickup_datetime'] = pd.to_datetime(data['tpep_pickup_datetime'] ,format='%Y-%m-%d %H:%M:%S')
data['tpep_dropoff_datetime'] = pd.to_datetime(data['tpep_dropoff_datetime'] ,format='%Y-%m-%d %H:%M:%S')

In [ ]:
data.dtypes

In [ ]:
data = data.sort_values(by=['tpep_pickup_datetime'])

# 2. EDA
1. Analisis de zonas 
2. Viajes por distrito

In [ ]:
import shapefile
from shapely.geometry import Polygon
from descartes.patch import PolygonPatch
import matplotlib as mpl

In [ ]:
# utilitly functions
def get_lat_lon(sf):
    content = []
    for sr in sf.shapeRecords():
        shape = sr.shape
        rec = sr.record
        loc_id = rec[shp_dic['LocationID']]
        
        x = (shape.bbox[0]+shape.bbox[2])/2
        y = (shape.bbox[1]+shape.bbox[3])/2
        
        content.append((loc_id, x, y))
    return pd.DataFrame(content, columns=["LocationID", "longitude", "latitude"])

def get_boundaries(sf):
    lat, lon = [], []
    for shape in list(sf.iterShapes()):
        lat.extend([shape.bbox[0], shape.bbox[2]])
        lon.extend([shape.bbox[1], shape.bbox[3]])

    margin = 0.01 # buffer to add to the range
    lat_min = min(lat) - margin
    lat_max = max(lat) + margin
    lon_min = min(lon) - margin
    lon_max = max(lon) + margin

    return lat_min, lat_max, lon_min, lon_max

def draw_region_map(ax, sf, heat={}):
    continent = [235/256, 151/256, 78/256]
    ocean = (89/256, 171/256, 227/256)    
    
    reg_list={'Staten Island':1, 'Queens':2, 'Bronx':3, 'Manhattan':4, 'EWR':5, 'Brooklyn':6}
    reg_x = {'Staten Island':[], 'Queens':[], 'Bronx':[], 'Manhattan':[], 'EWR':[], 'Brooklyn':[]}
    reg_y = {'Staten Island':[], 'Queens':[], 'Bronx':[], 'Manhattan':[], 'EWR':[], 'Brooklyn':[]}
    
    # colorbar
    if len(heat) != 0:
        norm = mpl.colors.Normalize(vmin=math.sqrt(min(heat.values())), vmax=math.sqrt(max(heat.values()))) #norm = mpl.colors.LogNorm(vmin=1,vmax=max(heat))
        cm=plt.get_cmap('Reds')
        #sm = plt.cm.ScalarMappable(cmap=cm, norm=norm)
        #sm.set_array([])
        #plt.colorbar(sm, ticks=np.linspace(min(heat.values()),max(heat.values()),8), \
        #             boundaries=np.arange(min(heat.values())-10,max(heat.values())+10,.1))
    
    ax.set_facecolor(ocean)
    for sr in sf.shapeRecords():
        shape = sr.shape
        rec = sr.record
        reg_name = rec[shp_dic['borough']]
        
        if len(heat) == 0:
            norm = mpl.colors.Normalize(vmin=1,vmax=6) #norm = mpl.colors.LogNorm(vmin=1,vmax=max(heat))
            cm=plt.get_cmap('Pastel1')
            R,G,B,A = cm(norm(reg_list[reg_name]))
            col = [R,G,B]
        else:
            R,G,B,A = cm(norm(math.sqrt(heat[reg_name])))
            col = [R,G,B]
            
        # check number of parts (could use MultiPolygon class of shapely?)
        nparts = len(shape.parts) # total parts
        if nparts == 1:
            polygon = Polygon(shape.points)
            patch = PolygonPatch(polygon, facecolor=col, alpha=1.0, zorder=2)
            ax.add_patch(patch)
        else: # loop over parts of each shape, plot separately
            for ip in range(nparts): # loop over parts, plot separately
                i0 = shape.parts[ip]
                if ip < nparts-1:
                    i1 = shape.parts[ip+1]-1
                else:
                    i1 = len(shape.points)

                polygon = Polygon(shape.points[i0:i1+1])
                patch = PolygonPatch(polygon, facecolor=col, alpha=1.0, zorder=2)
                ax.add_patch(patch)
                
        reg_x[reg_name].append((shape.bbox[0]+shape.bbox[2])/2)
        reg_y[reg_name].append((shape.bbox[1]+shape.bbox[3])/2)
        
    for k in reg_list:
        if len(heat)==0:
            plt.text(np.mean(reg_x[k]), np.mean(reg_y[k]), k, horizontalalignment='center', verticalalignment='center',
                        bbox=dict(facecolor='black', alpha=0.5), color="white", fontsize=12)     
        else:
            plt.text(np.mean(reg_x[k]), np.mean(reg_y[k]), "{}\n({}K)".format(k, heat[k]/1000), horizontalalignment='center', 
                     verticalalignment='center',bbox=dict(facecolor='black', alpha=0.5), color="white", fontsize=12)       

    # display
    limits = get_boundaries(sf)
    plt.xlim(limits[0], limits[1])
    plt.ylim(limits[2], limits[3])

def draw_zone_map(ax, sf, heat={}, text=[], arrows=[]):
    continent = [235/256, 151/256, 78/256]
    ocean = (89/256, 171/256, 227/256)
    theta = np.linspace(0, 2*np.pi, len(text)+1).tolist()
    ax.set_facecolor(ocean)
    
    # colorbar
    if len(heat) != 0:
        norm = mpl.colors.Normalize(vmin=min(heat.values()),vmax=max(heat.values())) #norm = mpl.colors.LogNorm(vmin=1,vmax=max(heat))
        cm=plt.get_cmap('Reds')
        sm = plt.cm.ScalarMappable(cmap=cm, norm=norm)
        sm.set_array([])
        plt.colorbar(sm, ticks=np.linspace(min(heat.values()),max(heat.values()),8),
                     boundaries=np.arange(min(heat.values())-10,max(heat.values())+10,.1))
    
    for sr in sf.shapeRecords():
        shape = sr.shape
        rec = sr.record
        loc_id = rec[shp_dic['LocationID']]
        zone = rec[shp_dic['zone']]
        
        if len(heat) == 0:
            col = continent
        else:
            if loc_id not in heat:
                R,G,B,A = cm(norm(0))
            else:
                R,G,B,A = cm(norm(heat[loc_id]))
            col = [R,G,B]

        # check number of parts (could use MultiPolygon class of shapely?)
        nparts = len(shape.parts) # total parts
        if nparts == 1:
            polygon = Polygon(shape.points)
            patch = PolygonPatch(polygon, facecolor=col, alpha=1.0, zorder=2)
            ax.add_patch(patch)
        else: # loop over parts of each shape, plot separately
            for ip in range(nparts): # loop over parts, plot separately
                i0 = shape.parts[ip]
                if ip < nparts-1:
                    i1 = shape.parts[ip+1]-1
                else:
                    i1 = len(shape.points)

                polygon = Polygon(shape.points[i0:i1+1])
                patch = PolygonPatch(polygon, facecolor=col, alpha=1.0, zorder=2)
                ax.add_patch(patch)
        
        x = (shape.bbox[0]+shape.bbox[2])/2
        y = (shape.bbox[1]+shape.bbox[3])/2
        if (len(text) == 0 and rec[shp_dic['Shape_Area']] > 0.0001):
            plt.text(x, y, str(loc_id), horizontalalignment='center', verticalalignment='center')            
        elif len(text) != 0 and loc_id in text:
            #plt.text(x+0.01, y-0.01, str(loc_id), fontsize=12, color="white", bbox=dict(facecolor='black', alpha=0.5))
            eta_x = 0.05*np.cos(theta[text.index(loc_id)])
            eta_y = 0.05*np.sin(theta[text.index(loc_id)])
            ax.annotate("[{}] {}".format(loc_id, zone), xy=(x, y), xytext=(x+eta_x, y+eta_y),
                        bbox=dict(facecolor='black', alpha=0.5), color="white", fontsize=12,
                        arrowprops=dict(facecolor='black', width=3, shrink=0.05))
    if len(arrows)!=0:
        for arr in arrows:
            ax.annotate('', xy = arr['dest'], xytext = arr['src'], size = arr['cnt'],
                    arrowprops=dict(arrowstyle="fancy", fc="0.6", ec="none"))
    
    # display
    limits = get_boundaries(sf)
    plt.xlim(limits[0], limits[1])
    plt.ylim(limits[2], limits[3])

In [ ]:
sf = shapefile.Reader('../input/taxi-zone/taxi_zones/taxi_zones.shp')
fields_name = [field[0] for field in sf.fields[1:]]
shp_dic = dict(zip(fields_name, list(range(len(fields_name)))))
attributes = sf.records()
shp_attr = [dict(zip(fields_name, attr)) for attr in attributes]

df_loc = pd.DataFrame(shp_attr).join(get_lat_lon(sf).set_index("LocationID"), on="LocationID")
df_loc.head(50)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,8))
ax = plt.subplot(1, 2, 1)
ax.set_title("Boroughs in NYC")
draw_region_map(ax, sf)
ax = plt.subplot(1, 2, 2)
ax.set_title("Zones in NYC")
draw_zone_map(ax, sf)

In [ ]:
zone_dict = dict(zip(df_loc.LocationID, df_loc.zone))
district_dict = dict(zip(df_loc.LocationID, df_loc.borough))

In [ ]:
zone_dict


In [ ]:
data['pickup_zona'] = data.PULocationID.map(zone_dict)
data['pickup_district'] = data.PULocationID.map(district_dict)
data['dropoff_zona'] = data.DOLocationID.map(zone_dict)
data['dropoff_district'] = data.DOLocationID.map(district_dict)

In [ ]:
data.head()

In [ ]:
def plot_histogram(title: str = '<No title specified>', 
                   field_names=None, xlabel_title='<Not specified>', ylabel_title='<Not specified>', 
                   xticks_label_rotation: int = None,
                   sort_values: bool = None, bins=10, width: int = 20.0, height: int = 5.0):
    plt.rcParams['figure.figsize'] = [width, height]
    if xticks_label_rotation:
        plt.xticks(rotation=xticks_label_rotation)
    plt.suptitle(title, fontsize="xx-large", fontstyle="normal")
    if sort_values:
        ax = data[field_names].sort_values(ascending=sort_values).hist(bins=bins)
    else:
        ax = data[field_names].hist(bins=bins)
    ax.set_xlabel(xlabel_title)
    ax.set_ylabel(ylabel_title)

def plot_value_counts(title: str = '<No title specified>', 
                      field_names=None, xlabel_title='<Not specified>', ylabel_title='<Not specified>',
                      kind='barh', value_filter_func=None,
                      width: int = 20.0, height: int = 5.0):
    plt.rcParams['figure.figsize'] = [width, height]
    value_counts_dropoff = data[field_names].value_counts()
    value_counts_dropoff = value_counts_dropoff.sort_values(ascending=True)
    if value_filter_func: 
        cutoff_filter = value_filter_func(value_counts_dropoff)
        ax = value_counts_dropoff[cutoff_filter].plot(kind=kind, title=title)
    else:
        ax = value_counts_dropoff.plot(kind=kind, title=title)
    
    ax.set_xlabel(xlabel_title)
    ax.set_ylabel(ylabel_title)

In [ ]:
plot_histogram("Distrito mas popular de salida", 'pickup_district', 
               xlabel_title='NYC Distritos',ylabel_title='Viajes realizados',)

In [ ]:
plot_histogram("Distrito mas popular de llegada", 'dropoff_district',
                ylabel_title='Viajes realizados', xlabel_title='NYC Distrito',)

In [ ]:
plot_value_counts("Zona mas popular de salida", 
                  xlabel_title='Viajes realizados', ylabel_title='NYC Vecindarios',
                  field_names=['PULocationID','pickup_zona', 'pickup_district'], 
                  value_filter_func=lambda x: x > 10_000,
                  width=15.0, height=20.0)

In [ ]:
plot_value_counts("Zona mas popular de llegada", 
                  xlabel_title='Viajes realizados', ylabel_title='NYC Distrito',
                  field_names=['dropoff_zona', 'dropoff_district'], 
                  value_filter_func=lambda x: x > 10_000,
                  width=15.0, height=15.0)

# 3. Data Cleaning
1. Viajes fuera de fecha
2. Numero de pasajeros
3. Distancia de los viajes
3. Duracion de los viajes 
4. Velocidad de los viajes

# Viajes fuera de fecha

Los viajes de salida deben estar comprendidos en el periodo de enero del 2019

In [ ]:
print(data["tpep_pickup_datetime"].max())
print(data["tpep_pickup_datetime"].min())


menor al periodo enero 2019

In [ ]:
data[data.tpep_pickup_datetime < '20190101']

In [ ]:

data_clean = data[data.tpep_pickup_datetime >= '20190101']

mayor al periodo enero 2019

In [ ]:
data_clean[data_clean.tpep_pickup_datetime >= '20190201']

In [ ]:
data_clean = data_clean[data_clean.tpep_pickup_datetime < '20190201']

In [ ]:
print(data_clean["tpep_pickup_datetime"].max())
print(data_clean["tpep_pickup_datetime"].min())

habian 441 datos por debajo del periodo enero 2019 y 96 datos por arriba del periodo

In [ ]:
def boxplot(columna,label,data):
    fig = plt.figure(figsize = (10,6))
    ax = sns.boxplot(columna, data = data, orient = "v")

    plt.tick_params(labelsize = 20)
    plt.ylabel(label, fontsize = 20)
    plt.show()

# Numero de pasajeros
La cantidad máxima de pasajeros permitidos en un taxi amarillo por ley es de cuatro (4) en un taxi de cuatro (4) pasajeros o de cinco (5) pasajeros en un taxi de cinco (5) pasajeros, excepto que se debe aceptar un pasajero adicional si el pasajero es menor de siete (7) años y está sostenido en el regazo de un pasajero adulto sentado en la parte trasera ".

Los viajes con 0 pasajeros son considerados por omision de dato y no deberia eliminarse. 

In [ ]:
data_clean.passenger_count.describe()

In [ ]:
boxplot('passenger_count','passenger_count',data_clean)

In [ ]:
data_clean = data_clean[data_clean.passenger_count<=7]
boxplot('passenger_count','passenger_count',data_clean)

# Distancia de los viajes

La distancia mas larga que se puede recorrer entre los distritos mas alejados es de 40 millas, lo que indica que no se debe sobrepasar dichos limites.
La distancia de los viajes deben ser mayor a cero. Sino seria un error.

In [ ]:
data_clean.trip_distance.describe()

In [ ]:
boxplot('trip_distance','Trip Distance(Millas)',data_clean)

In [ ]:
data_clean = data_clean[(data_clean.trip_distance>0.0)&(data_clean.trip_distance<40.0)]

boxplot('trip_distance','Trip Distance(Millas)',data_clean)

# Duracion y velocidad de los viajes
Al menos un viaje debe durar 30 segundos para ser considerado sino seria un error. Tambien los viajes permitidos no deben sobresar las 12 horas segun reglamento. 
Se crean dos columnas : 
1. Trip_duration el cual es la duracion del viaje en minutos
2. speed es la velocidad promedio del viaje expresado en millas por hora

In [ ]:
data_clean["trip_duration"] = (data_clean["tpep_dropoff_datetime"] - data_clean["tpep_pickup_datetime"])/ np.timedelta64(1, 'm')
data_clean["speed"] = (data_clean["trip_distance"]/data_clean["trip_duration"])*60  #speed  millas/hr


In [ ]:
boxplot('trip_duration','trip_duration(minutos)',data_clean)

In [ ]:
data_clean = data_clean[(data_clean.trip_duration > 0.5 )&(data_clean.trip_duration < 12*60)]
boxplot('trip_duration','trip_duration(minutos)',data_clean)

La velocidad maxima permitida es variable , pero debe ser fisicamente posible en los limites de 50 millas por hora para una ciudad transitada

In [ ]:
boxplot('speed','speed(millas por hora)',data_clean)

In [ ]:
quantile_speed = data_clean.speed.quantile(np.round(np.arange(0.00, 1.01, 0.01), 2))
print(quantile_speed)

Existe  0.01% de viajes que tienen velocidades irrazonables, por lo tanto son errores de medicion

In [ ]:
data_clean = data_clean[(data_clean.speed > 0 )&(data_clean.speed < 40)]
boxplot('speed','speed(millas por hora)',data_clean)

# Costo por viaje

¿Cómo se calculan las tarifas estándar?

Cargo inicial de $ 2.50.

Más 50 centavos por 1/5 de milla cuando viaja a más de 12 mph o por 60 segundos en tráfico lento o cuando el vehículo está parado.

Más un recargo de 50 centavos por noche de 8 p.m. a 6 a.m.

Más un recargo de $ 1.00 por hora punta de 4:00 p.m. a 8:00 p.m. los días de semana, excepto los días festivos.

Más un recargo por congestión del estado de Nueva York de $ 2.50 

Más propinas y peajes.

No hay cargo por pasajeros adicionales, equipaje o maletas, ni pago con tarjeta de crédito.

El mensaje de tarifa en pantalla debe leer: "Tarifa n. ° 01 - Tarifa estándar de la ciudad".

Asegúrese de llevar siempre su recibo.



In [ ]:
boxplot('fare_amount','costo viaje',data_clean)

In [ ]:
quantile_costo = data_clean.fare_amount.quantile(np.round(np.arange(0.00, 1.01, 0.01), 2))
print(quantile_costo)

Removemos los viajes que tienen costo menor a 2.5 dolares por ser el cargo inicial y los pagos maximos estan por los 52 dolares.

In [ ]:
data_clean = data_clean[(data_clean.fare_amount >= 2.5)&(data_clean.fare_amount < 55)]
boxplot('fare_amount','costo viaje',data_clean)

# Agregacion de datos
1. Resampling por hora

In [ ]:
columns = ['tpep_pickup_datetime',
            'PULocationID',

          ]
data = data_clean[columns]
#data = data.set_index("tpep_pickup_datetime")
#data_resample =data.resample('1H').count()

In [ ]:
data.head()

In [ ]:
data =data.groupby('PULocationID').resample('D', on='tpep_pickup_datetime').count()
data = data.drop("tpep_pickup_datetime", axis=1)
data = data.unstack('PULocationID')

In [ ]:
plt.figure(figsize=(30,15))
heat_map = sns.heatmap(data, yticklabels=False)

# Seleccionando la zona

In [ ]:
columns2 = ['tpep_pickup_datetime',
            'PULocationID',
            'pickup_district'
          ]

data_manhattan = data_clean[columns2]
data_manhattan = data_manhattan[data_manhattan.pickup_district=='Manhattan']
data_manhattan = data_manhattan.drop('pickup_district', axis=1)

In [ ]:
data_manhattan


Resampling por dia

In [ ]:
data_manhattanxdia = data_manhattan.groupby('PULocationID').resample('D', on='tpep_pickup_datetime').count()
data_manhattanxdia = data_manhattanxdia.drop("tpep_pickup_datetime", axis=1)
data_manhattanxdia = data_manhattanxdia.unstack('PULocationID')

In [ ]:
data_manhattanxdia

In [ ]:
plt.figure(figsize=(30,15))
sns.heatmap(data_manhattanxdia, yticklabels=False)

In [ ]:
plt.figure(figsize=(30,15))
zonas_test= [237,236,161,162] # zonas con mas movimiento
for i in zonas_test:
    plt.plot(data_manhattanxdia['PULocationID'][i], label=i)
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left', ncol=len(zonas_test), mode="expand", borderaxespad=0. , prop={"size":20})

La actividad de taxis disminuye en los fines de semana.
Enero tuvo 4 fines de semana:
Se observan 4 valles que corresponden a los fines de semanas.

# resampling por  hora

In [ ]:
data_manhattanxhora =data_manhattan.groupby('PULocationID').resample('H', on='tpep_pickup_datetime').count()
data_manhattanxhora = data_manhattanxhora.drop("tpep_pickup_datetime", axis=1)
data_manhattanxhora = data_manhattanxhora.unstack('PULocationID')
data_manhattanxhora.reset_index()

In [ ]:
plt.figure(figsize=(30,15))
zonas_test= [237,236,161,162]
for i in zonas_test:
    plt.plot(data_manhattanxhora['PULocationID'][i], label=i)
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left', ncol=len(zonas_test), mode="expand", borderaxespad=0. , prop={"size":20})

El mes de enero tiene 31 dias
Se pueden ver 31 picos que corresponden a las horas activas (horas laborales) y los valles corresponden a las horas inactivas (horas de descanso)

# Resampling cada 10 minutos

In [ ]:
data_manhattanx10min =data_manhattan.groupby('PULocationID').resample('10Min', on='tpep_pickup_datetime').count()
data_manhattanx10min = data_manhattanx10min.drop("tpep_pickup_datetime", axis=1)
data_manhattanx10min = data_manhattanx10min.unstack('PULocationID')
data_manhattanx10min.reset_index()

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(data_manhattanx10min.isna().sum()['PULocationID'])

In [ ]:
data_manhattanx10min.columns[data_manhattanx10min.isna().any()].tolist()

In [ ]:
data_manhattanx10min= data_manhattanx10min.dropna(axis=1)

In [ ]:
plt.figure(figsize=(30,15))
zonas_test= [237,236,161,162]
for i in zonas_test:
    plt.plot(data_manhattanx10min['PULocationID'][i], label=i)
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left', ncol=len(zonas_test), mode="expand", borderaxespad=0. , prop={"size":20})

# Modelado

In [ ]:
import tensorflow as tf

from sklearn.preprocessing import RobustScaler
series = data_manhattanxhora['PULocationID'][237].to_numpy()
series = series.reshape((len(series), 1))
transformer = RobustScaler()
transformer = transformer.fit(series)
series =transformer.transform(series)

In [ ]:
series = data_manhattanx10min['PULocationID'][237].to_numpy()
plt.figure(figsize=(30,15))
plt.plot(series, label = "serie")

In [ ]:
split_time = 3500
time = np.arange(len(series), dtype="float32")
time_train = time[:split_time]
x_train = series[:split_time]
time_test = time[split_time:]
x_test = series[split_time:]

plt.figure(figsize=(30,15))
plt.plot(time_train,x_train, label = "train")
plt.plot(time_test,x_test, label = "test")
plt.legend()

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    #series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[-1]))
    return ds.batch(batch_size).prefetch(1)

In [ ]:
window_size = 5
batch_size = 2
shuffle_buffer_size = 1000
window=windowed_dataset(series, window_size, batch_size, shuffle_buffer_size)
print(window)
for x, y in window:
  print("x " ,x.numpy()," y ", y.numpy())

# Modelo 1 : Redes Densas

In [ ]:
window_size = 7
batch_size = 64
tf.keras.backend.clear_session()
dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_shape=[window_size])
])


model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(lr=1e-5, momentum=0.9),metrics=["mae"])
history = model.fit(dataset,epochs=20)

plt.figure(figsize=(6,4))
plt.plot(history.history["loss"])

In [ ]:
forecast = []
results = []
for time in range(len(time_test)):
  #print(series[time:time + window_size][np.newaxis] , " --> ",model.predict(series[time:time + window_size][np.newaxis]))
    forecast.append(model.predict(series[split_time+time-window_size:split_time+time][np.newaxis]))

#forecast = forecast[split_time-window_size:]
results = np.array(forecast)[:, 0, 0]

plt.plot(results, label ="pronostico")
plt.plot(x_test, label = "serie")
plt.legend()

# Modelo 2 : Redes Recurrentes GRU

In [ ]:
window_size = 7
batch_size = 64
tf.keras.backend.clear_session()
dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

model = tf.keras.models.Sequential([
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),input_shape=[None]),
    tf.keras.layers.GRU(64),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x:x*100)
])


model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9),metrics=["mae"])
history = model.fit(dataset,epochs=30)

plt.figure(figsize=(6,4))
plt.plot(history.history["loss"])

In [ ]:
forecast = []
results = []
for time in range(len(time_test)):
  #print(series[time:time + window_size][np.newaxis] , " --> ",model.predict(series[time:time + window_size][np.newaxis]))
    forecast.append(model.predict(series[split_time+time-window_size:split_time+time][np.newaxis]))

#forecast = forecast[split_time-window_size:]
results = np.array(forecast)[:, 0, 0]

plt.plot(results, label ="pronostico")
plt.plot(x_test, label = "serie")
plt.legend()

In [ ]:
series[split_time+time-window_size:split_time+time][np.newaxis]

# Modelo 3 : Redes Recurrentes  LSTM

In [ ]:
window_size = 7
batch_size = 64
tf.keras.backend.clear_session()
dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
#(batch_size,timestamp, series_dimensionality)
model = tf.keras.models.Sequential([
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),input_shape=[None]),
    #tf.keras.layers.LSTM(100, return_sequences=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x:x*200)
])


model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9),metrics=["mae"])
history = model.fit(dataset,epochs=30)

plt.figure(figsize=(6,4))
plt.plot(history.history["loss"])

In [ ]:
forecast = []
results = []
for time in range(len(time_test)):
  #print(series[time:time + window_size][np.newaxis] , " --> ",model.predict(series[time:time + window_size][np.newaxis]))
    forecast.append(model.predict(series[split_time+time-window_size:split_time+time][np.newaxis]))

#forecast = forecast[split_time-window_size:]
results = np.array(forecast)[:, 0, 0]
plt.plot(results, label ="pronostico")
plt.plot(x_test, label = "serie")
plt.legend()

In [ ]:
tf.keras.metrics.mean_absolute_error(x_test, results).numpy()

# Modelo 4 : Redes Recurrentes BiLSTM

Modelo 4 : BiLSTM

In [ ]:
window_size = 7
batch_size = 64
shuffle_buffer_size = 1000

model = tf.keras.models.Sequential([
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),input_shape=[None]),
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x:x*200)
])


model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9),metrics=["mae"])
history = model.fit(dataset,epochs=30)

plt.figure(figsize=(6,4))
plt.plot(history.history["loss"])

In [ ]:
forecast = []
results = []
for time in range(len(time_test)):
  #print(series[time:time + window_size][np.newaxis] , " --> ",model.predict(series[time:time + window_size][np.newaxis]))
    forecast.append(model.predict(series[split_time+time-window_size:split_time+time][np.newaxis]))

#forecast = forecast[split_time-window_size:]
results = np.array(forecast)[:, 0, 0]

plt.plot(results, label ="pronostico")
plt.plot(x_test, label = "serie")
plt.legend()


In [ ]:
tf.keras.metrics.mean_absolute_error(x_test, results).numpy()

# Modelo 5 : Red hibrida ( CONV+LSTM )

In [ ]:
def windowed_dataset2(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)


window_size = 7
batch_size = 64
shuffle_buffer_size = 1000
dataset2 = windowed_dataset2(x_train, window_size, batch_size, shuffle_buffer_size)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=3,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[None, 1]),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x:x*200)
])


model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9),metrics=["mae"])
history = model.fit(dataset2,epochs=30)

plt.figure(figsize=(6,4))
plt.plot(history.history["loss"])

forecast = []
results = []
for time in range(len(time_test)):
  #print(series[time:time + window_size][np.newaxis] , " --> ",model.predict(series[time:time + window_size][np.newaxis]))
    forecast.append(model.predict(series[split_time+time-window_size:split_time+time][np.newaxis]))

#forecast = forecast[split_time-window_size:]
results = np.array(forecast)[:, 0, 0]

plt.plot(results, label ="pronostico")
plt.plot(x_test, label = "serie")
plt.legend()

Como resultados se tiene:
Para 64 nodos

# Multivariable

# Para 2 entradas y 2 salidas

In [ ]:
series1 = data_manhattanx10min['PULocationID'][237].to_numpy()[np.newaxis].transpose()
series2 = data_manhattanx10min['PULocationID'][90].to_numpy()[np.newaxis].transpose()
serie_multi = np.append(series1, series2,axis=1)
plt.figure(figsize=(30,15))
plt.plot(serie_multi, label = "serie")


x = np.array([[10,20,30]]).transpose()
y = np.array([[100, 200,300]]).transpose()
print(np.append(x, y, axis=1))

In [ ]:
split_time = 3500
time = np.arange(len(series1), dtype="float32")
time_train = time[:split_time]
x_train_multi = serie_multi[:split_time]
time_test = time[split_time:]
x_test_multi = serie_multi[split_time:]

plt.figure(figsize=(30,15))
plt.plot(time_train,x_train_multi, label = "train")
plt.plot(time_test,x_test_multi, label = "test")
plt.legend()

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    #series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[-1]))
    return ds.batch(batch_size).prefetch(1)

In [ ]:
window_size = 5
batch_size = 2
shuffle_buffer_size = 1000
window=windowed_dataset(serie_multi, window_size, batch_size, shuffle_buffer_size)
print(window)
for x, y in window:
  print("x " ,x.numpy()," y ", y.numpy())

In [ ]:
window_size = 7
batch_size = 64
n_features = 2
tf.keras.backend.clear_session()
dataset_multi = windowed_dataset(x_train_multi, window_size, batch_size, shuffle_buffer_size)
#(batch_size,timestamp, n_features)
model = tf.keras.models.Sequential([
    #tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),input_shape=[None]),
    #tf.keras.layers.LSTM(100, return_sequences=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(n_features), #n_features
    tf.keras.layers.Lambda(lambda x:x*100)
])


model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9),metrics=["mae"])
history = model.fit(dataset_multi,epochs=20)

plt.figure(figsize=(6,4))
plt.plot(history.history["loss"])

In [ ]:
forecast = []
results = []
for time in range(len(time_test)):
  #print(series[time:time + window_size][np.newaxis] , " --> ",model.predict(series[time:time + window_size][np.newaxis]))
    forecast.append(model.predict(serie_multi[split_time+time-window_size:split_time+time][np.newaxis]))

results = np.array(forecast)[:, 0, :]
plt.plot(results, label ="pronostico")
plt.plot(x_test_multi, label = "serie")
plt.legend()


In [ ]:
window_size = 7
batch_size = 64
shuffle_buffer_size = 1000

model = tf.keras.models.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(2),
    tf.keras.layers.Lambda(lambda x:x*200)
])


model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9),metrics=["mae"])
history = model.fit(dataset_multi,epochs=30)

plt.figure(figsize=(6,4))
plt.plot(history.history["loss"])

In [ ]:
forecast = []
results = []
for time in range(len(time_test)):
    forecast.append(model.predict(serie_multi[split_time+time-window_size:split_time+time][np.newaxis]))

results = np.array(forecast)[:, 0, :]
plt.plot(results, label ="pronostico")
plt.plot(x_test_multi, label = "serie")
plt.legend()

# Para todos las zonas

serie_multi_manhattan= data_manhattanx10min['PULocationID'][4].to_numpy()[np.newaxis].transpose()

for i in data_manhattanx10min['PULocationID'].columns:
    if i != 4:
        serie_i = data_manhattanx10min['PULocationID'][i].to_numpy()[np.newaxis].transpose()
        serie_multi_manhattan = np.append(serie_multi_manhattan, serie_i,axis=1)

In [ ]:
serie_multi_manhattan=[]
serie_multi_manhattan= data_manhattanx10min['PULocationID'][4].to_numpy()[np.newaxis].transpose()
zonas_manhattan = data_manhattanx10min['PULocationID'].columns

for i in zonas_manhattan[1:]:
    serie_i = data_manhattanx10min['PULocationID'][i].to_numpy()[np.newaxis].transpose()
    serie_multi_manhattan = np.append(serie_multi_manhattan, serie_i,axis=1)


In [ ]:
serie_multi_manhattan

In [ ]:
serie_multi_manhattan = []
#zonas_mas_transitadas = [237,236,161,162,186,48,170, 234,142,239]
zonas_mas_transitadas = [237,236,161]
serie_multi_manhattan = data_manhattanx10min['PULocationID'][237].to_numpy()[np.newaxis].transpose()
for i in zonas_mas_transitadas[1:]:
    serie_i = data_manhattanx10min['PULocationID'][i].to_numpy()[np.newaxis].transpose()
    serie_multi_manhattan = np.append(serie_multi_manhattan, serie_i,axis=1)

In [ ]:
serie_multi_manhattan

In [ ]:
split_time = 3500
time = np.arange(len(series1), dtype="float32")
time_train = time[:split_time]
x_train_multi_manhattan = serie_multi_manhattan[:split_time]
time_test = time[split_time:]
x_test_multi_manhattan = serie_multi_manhattan[split_time:]

plt.figure(figsize=(30,15))
#plt.plot(time_train,x_train_multi, label = "train")
plt.plot(time_test,x_test_multi_manhattan, label = "test")
plt.legend()

In [ ]:
window_size = 7
batch_size = 64
n_features = len(zonas_manhattan)

tf.keras.backend.clear_session()
dataset_multi_manhattan = windowed_dataset(x_train_multi_manhattan, window_size, batch_size, shuffle_buffer_size)
#(batch_size,timestamp, n_features)
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(n_features), #n_features
    tf.keras.layers.Lambda(lambda x:x*200)
])


model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9),metrics=["mae"])
history = model.fit(dataset_multi_manhattan,epochs=30)

plt.figure(figsize=(6,4))
plt.plot(history.history["loss"])

# Prediccion del ultimo dia para todas las zonas

In [ ]:
tiempo_prediccion = len(time)-1
test = serie_multi_manhattan[tiempo_prediccion]
prediccion = model.predict(serie_multi_manhattan[tiempo_prediccion-window_size:tiempo_prediccion-1][np.newaxis])

In [ ]:
index = []
for i in zonas_manhattan:
    index.append(zone_dict[i])
df = pd.DataFrame({'speed': test,'lifespan': prediccion}, index=index)
ax = df.plot.bar()